# ISODISTORT Automation
by Nicholas Wagner 2016-03-21

We begin by creating a set of parent structures to be used with ISODISTORT from our starting cifs

In [1]:
# Test case: assume we have parent structures already in a directory
# from pymatgen import * # Used to edit structure files
from os import listdir
from os.path import isfile, join, abspath


# Get list of paths to parent cif files for feeding into ISODISTORT
onlyparents = [f for f in listdir("./") if isfile(join("./", f)) and 'parent' in f]
parent_paths = []
for element in onlyparents:
    parent_paths.append(abspath(element))
parent_paths.sort()    
    
    
onlychildren = [f for f in listdir("./") if isfile(join("./", f)) and 'parent' not in f and 'cif' in f]
child_paths = []
for element in onlychildren:
    child_paths.append(abspath(element))
child_paths.sort()

## Next, we upload our parent structure to ISODISTORT and select "OK"

In [2]:
### Use pyautogui to perform clicking and keyboard actions
import pyautogui, sys
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

# First page where you upload parent
pyautogui.PAUSE = 0.4
browser = webdriver.Firefox() # Get local session of firefox
browser.get("http://stokes.byu.edu/iso/isodistort.php") # Load page
actions = ActionChains(browser)
select_parent_button = browser.find_element_by_name("toProcess")
select_parent_button.click()

# File dialog on first page
pyautogui.hotkey('ctrl', 'l')
pyautogui.typewrite(parent_paths[0])
pyautogui.press("enter")
elem = browser.find_element_by_class_name("btn.btn-primary")
time.sleep(0.2)
elem.click()

In [3]:
# Second page where you upload child cif
time.sleep(3)
select_child_button = browser.find_element_by_name("toProcess")
select_child_button.click()
time.sleep(0.5)
pyautogui.typewrite(child_paths[0])
pyautogui.press("enter")
time.sleep(0.5)
elems = browser.find_elements_by_class_name("btn-primary")
window_before = browser.window_handles[0]
elems[6].click()
time.sleep(3)

In [4]:
# Third page where you select bases
window_after=browser.window_handles[1]
browser.close()
browser.switch_to_window(window_after)
bases = browser.find_element_by_name("basisselect")
bases.click()
pyautogui.press("down")
pyautogui.press("enter")
ok_button = browser.find_element_by_class_name("btn-primary")
ok_button.click()
time.sleep(2)

In [5]:
# Final page. Time to get mode details
radio_buttons = browser.find_elements_by_name("origintype")
radio_buttons[6].click()
ok_button = browser.find_element_by_class_name("btn-primary")
ok_button.click()
time.sleep(3)
browser.switch_to.window(browser.window_handles[1])
html = browser.page_source
browser.quit()

In [6]:
# Scrape mode details
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "lxml")

In [7]:
# Move through html to find mode amplitudes
found_text = []
for i, x in enumerate(soup.find("b").nextSiblingGenerator()):
    if i == 20:
        found_text.append(x)
    elif 'b/' in x:
        i+=1
        break
    else:
        continue
found_text = found_text[0].split("\n")
for x in found_text:
    print(x)


[1/2,1/2,1/2]R3+[O1:d:dsp]A2u(a)    0.09401   0.04700  0.03324
[1/2,1/2,1/2]R3+  all               0.09401  0.04700
 
[1/2,1/2,1/2]R4+[O1:d:dsp]Eu(a)    -1.29378  -0.64689  0.45742
[1/2,1/2,1/2]R4+[O1:d:dsp]Eu(b)    -0.00894  -0.00447  0.00316
[1/2,1/2,1/2]R4+  all               1.29381  0.64691
 
[1/2,1/2,1/2]R5+[La1:b:dsp]T1u(a)  -0.11706  -0.05853  0.05853
[1/2,1/2,1/2]R5+[O1:d:dsp]Eu(a)     0.04720   0.02360  0.01669
[1/2,1/2,1/2]R5+  all               0.12622  0.06311
 
[0,1/2,0]X1+[La1:b:dsp]T1u(a)      -0.00204  -0.00102  0.00102
[0,1/2,0]X1+[O1:d:dsp]A2u(a)        0.00236   0.00118  0.00118
[0,1/2,0]X1+  all                   0.00312  0.00156
 
[0,1/2,0]X5+[La1:b:dsp]T1u(a)       0.48010   0.24005  0.24005
[0,1/2,0]X5+[O1:d:dsp]Eu(a)         0.17247   0.08624  0.08623
[0,1/2,0]X5+  all                   0.51014  0.25507
 
[1/2,1/2,0]M2+[O1:d:dsp]A2u(a)      0.00550   0.00275  0.00194
[1/2,1/2,0]M2+  all                 0.00550  0.00275
 
[1/2,1/2,0]M3+[O1:d:dsp]Eu(a)       0.8

In [8]:
# Strip out text before irrep label
for i, x in enumerate(found_text):
    if ']' in found_text[i]:
        found_text[i] = ''.join(x.split("]")[1:])
    found_text[i] = found_text[i].split(" ")
    found_text[i] = list(filter(None, found_text[i]))
    for j in range(len(found_text[i])):
        if '[' in found_text[i][j]:
            temp = found_text[i][j].split("[")
            found_text[i].insert(1, temp[1])
            found_text[i][j] = temp[0]
found_text = [x for x in found_text if x != []]

In [9]:
# Gather only 'all' mode amplitude in parent setting for each mode
mode_amps = {}
for x in found_text:
    if 'all' in x[1]:
        mode_amps[x[0]] = {'Ap': float(x[3])}
print(mode_amps)

{'R3+': {'Ap': 0.047}, 'R4+': {'Ap': 0.64691}, 'R5+': {'Ap': 0.06311}, 'X1+': {'Ap': 0.00156}, 'X5+': {'Ap': 0.25507}, 'M2+': {'Ap': 0.00275}, 'M3+': {'Ap': 0.4426}, 'M5+': {'Ap': 0.00322}}


In [10]:
import pandas as pd
df = pd.DataFrame.from_dict(mode_amps)
df

,M2+,M3+,M5+,R3+,R4+,R5+,X1+,X5+
Ap,0.00275,0.4426,0.00322,0.047,0.64691,0.06311,0.00156,0.25507
